22H1120016 - Trần Đăng Nam

Gom cụm văn bản với K-Means trên 20 Newsgroups Dataset

Dùng dataset: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html

# Mục tiêu:
1. Nạp dữ liệu từ 3 nhóm tin cụ thể: 'sci.space', 'soc.religion.christian', 'talk.politics.guns'.

2. Tiền xử lý và vector hóa văn bản bằng TF-IDF.

3. Áp dụng thuật toán K-Means để gom cụm các tài liệu thành 3 nhóm.

4. Đánh giá chất lượng gom cụm bằng các độ đo phổ biến.

In [2]:
import numpy as np
from time import time
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn import metrics
import logging

# Cấu hình logging để xem thông tin (tùy chọn)
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

In [4]:
# 2. Nạp dữ liệu (3 lớp theo yêu cầu)
# `fetch_20newsgroups` sẽ tải dữ liệu nếu chưa có và lưu vào cache.
# Tham số `remove` giúp loại bỏ các phần không mong muốn như header, footer, quote để tập trung vào nội dung chính.

# Định nghĩa các nhóm tin cần lấy
categories = [
    'sci.space',
    'soc.religion.christian',
    'talk.politics.guns',
]

print("Đang nạp dữ liệu 20 newsgroups cho các nhóm tin:")
print(categories)

# Nạp dữ liệu, chỉ lấy phần 'all' (bao gồm cả train và test)
dataset = fetch_20newsgroups(
    subset='all',  # Lấy cả tập train và test gộp lại
    categories=categories,
    shuffle=True,  # Mix dữ liệu để tạo random
    random_state=42, 
    remove=('headers', 'footers', 'quotes') # Loại bỏ các thứ k dùng - chỉ lấy nội dung 
)

print(f"{len(dataset.data)} tài liệu được nạp.")
print(f"{len(dataset.target_names)} nhóm tin.")

# dataset.data: list chứa nội dung các tài liệu (văn bản)
# dataset.target: numpy array chứa chỉ số của nhóm tin thật sự của mỗi tài liệu (0, 1, 2)
# dataset.target_names: list tên các nhóm tin tương ứng với chỉ số target

# Lưu lại nhãn thật để đánh giá sau này
labels_true = dataset.target
n_clusters_true = np.unique(labels_true).shape[0] # Số lượng nhóm tin thật sự (phải là 3)

print(f"Số lượng cụm thực tế (số nhóm tin): {n_clusters_true}")


2025-03-30 14:18:40,441 INFO Downloading 20news dataset. This may take a few minutes.
2025-03-30 14:18:40,444 INFO Downloading dataset from https://ndownloader.figshare.com/files/5975967 (14 MB)


Đang nạp dữ liệu 20 newsgroups cho các nhóm tin:
['sci.space', 'soc.religion.christian', 'talk.politics.guns']
2894 tài liệu được nạp.
3 nhóm tin.
Số lượng cụm thực tế (số nhóm tin): 3


K-Means hoạt động trên dữ liệu số. Chúng ta cần chuyển đổi văn bản thành các vector số.
TF-IDF (Term Frequency-Inverse Document Frequency) là một kỹ thuật phổ biến để làm điều này.
- **TF (Term Frequency):** Tần suất xuất hiện của một từ trong một tài liệu.
- **IDF (Inverse Document Frequency):** Độ đo mức độ quan trọng của một từ. Từ nào xuất hiện trong nhiều tài liệu sẽ có IDF thấp và ngược lại.


In [5]:
# 3. Vector hóa văn bản bằng TF-IDF

print("Trích xuất đặc trưng bằng TF-IDF Vectorizer...")
t0 = time()

vectorizer = TfidfVectorizer(
    max_df=0.5, # Bỏ từ xuất hiện > 50% tài liệu
    min_df=2,   # Bỏ từ xuất hiện < 2 tài liệu
    stop_words="english",
    use_idf=True
)

# Học từ vựng và biến đổi dữ liệu văn bản thành ma trận TF-IDF
X = vectorizer.fit_transform(dataset.data)

print(f"Hoàn thành trong {time() - t0:.3f}s")
print(f"Số lượng mẫu (tài liệu): {X.shape[0]}, Số lượng đặc trưng (từ): {X.shape[1]}")
print(f"Kích thước ma trận TF-IDF: {X.shape}")

Trích xuất đặc trưng bằng TF-IDF Vectorizer...
Hoàn thành trong 0.366s
Số lượng mẫu (tài liệu): 2894, Số lượng đặc trưng (từ): 15778
Kích thước ma trận TF-IDF: (2894, 15778)


In [6]:
# 4. Áp dụng K-Means Clustering

# Đặt số cụm bằng số nhóm tin đã biết
k = n_clusters_true

print(f"\nBắt đầu gom cụm với K-Means (k={k})...")
t0 = time()

km = KMeans(
    n_clusters=k,
    init="k-means++", # Khởi tạo cụm thông minh
    max_iter=300,    # Số vòng lặp tối đa
    n_init=10,       # Chạy 10 lần với các khởi tạo khác nhau, chọn lần tốt nhất
    random_state=42  # Để kết quả ổn định
)

# Thực hiện gom cụm trên dữ liệu TF-IDF
km.fit(X)

print(f"Hoàn thành trong {time() - t0:.3f}s")

# Lấy nhãn cụm được dự đoán cho từng tài liệu
labels_pred = km.labels_


Bắt đầu gom cụm với K-Means (k=3)...
Hoàn thành trong 0.412s


In [7]:
# 5. Đánh giá kết quả gom cụm

print("\nĐánh giá kết quả gom cụm:")
print(f"Homogeneity: {metrics.homogeneity_score(labels_true, labels_pred):.3f}")
print(f"Completeness: {metrics.completeness_score(labels_true, labels_pred):.3f}")
print(f"V-measure: {metrics.v_measure_score(labels_true, labels_pred):.3f}")
print(f"Adjusted Rand Index (ARI): {metrics.adjusted_rand_score(labels_true, labels_pred):.3f}")
print(f"Adjusted Mutual Information (AMI): {metrics.adjusted_mutual_info_score(labels_true, labels_pred):.3f}")

# Silhouette Score cần ma trận dữ liệu X và nhãn dự đoán
# Có thể chậm với dữ liệu lớn/chiều cao
print(f"Silhouette Coefficient: {metrics.silhouette_score(X, labels_pred, sample_size=1000):.3f}") # Lấy mẫu 1000 điểm để tính nhanh hơn

print(f"\nInertia (Tổng bình phương khoảng cách đến tâm cụm): {km.inertia_:.3f}")



Đánh giá kết quả gom cụm:
Homogeneity: 0.391
Completeness: 0.471
V-measure: 0.428
Adjusted Rand Index (ARI): 0.280
Adjusted Mutual Information (AMI): 0.427
Silhouette Coefficient: 0.010

Inertia (Tổng bình phương khoảng cách đến tâm cụm): 2748.963
